In [ ]:
!pip install transformers
!pip install SentencePiece
!pip install transformers[torch]
!pip install tweepy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 9.0 MB/s eta 0:00:00


In [ ]:
import csv
from pandas import ExcelWriter
import re
import pandas as pd

In [ ]:
# Membaca file Excel

df = pd.read_excel('dataset.xlsx', usecols=['content'])
df

,content
0,[APA SAJA KEUNIKAN DARI UNPAD DAN JATINANGOR?]...
1,Nek ono kuliah luring ngene tapi aku ngantuk b...
2,"banyak kuliah luring ngeluh, banyak kuliah dar..."
3,@collegemenfess Aku dulunya sih kos khusus cew...
4,@yjnluvv baiklah :( besok ada kuliah luring ga...
...,...
3944,Kuliah daring kok malah lebih banyak ya tugasn...
3945,"Kuliah media daring,\nTidak senikmat kuliah lu..."
3946,"selama daring, waktunya beberes hal-hal yang d..."
3947,"Kalo menurut sy, ketika model kuliah luring di..."


In [ ]:
from bs4 import BeautifulSoup

cleanHTML = []
i=0
for column in df.content:
    clean = BeautifulSoup(df.content[i], 'lxml') # menghilangkan text field as ‘&amp’,’&quot’,dll
    cleanHTML.insert(i, clean.get_text())
    i+=1

df["cleanHTML"] = cleanHTML

import re
cleanMention = []
i=0
for column in df.cleanHTML:
    clean = re.sub(r'@[A-Za-z0-9_]+', '', df.cleanHTML[i])
    clean = re.sub('https?://[A-Za-z0-9./]+','', clean)
    clean = re.sub(r"\\x(.){2}", "", clean) #menghilangkan kata belebih
    clean = re.sub(r"^b[\'\"]|#[A-Za-z0-9]+|RT|\\n|  +|:\(|:\)|:v|:V|:'\)|:'\(", " ", clean) # menghilangkan karakter lain
    clean = re.sub("\W+| +", " ", clean)
    clean = clean.lower()
    cleanMention.insert(i, clean)
    i += 1

df["cleanData"] = cleanMention
df

<ipython-input-6-694d5e1fa007>:6: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  clean = BeautifulSoup(df.content[i], 'lxml') # menghilangkan text field as ‘&amp’,’&quot’,dll


,content,cleanHTML,cleanData
0,[APA SAJA KEUNIKAN DARI UNPAD DAN JATINANGOR?]...,[APA SAJA KEUNIKAN DARI UNPAD DAN JATINANGOR?]...,apa saja keunikan dari unpad dan jatinangor h...
1,Nek ono kuliah luring ngene tapi aku ngantuk b...,Nek ono kuliah luring ngene tapi aku ngantuk b...,nek ono kuliah luring ngene tapi aku ngantuk b...
2,"banyak kuliah luring ngeluh, banyak kuliah dar...","banyak kuliah luring ngeluh, banyak kuliah dar...",banyak kuliah luring ngeluh banyak kuliah dari...
3,@collegemenfess Aku dulunya sih kos khusus cew...,@collegemenfess Aku dulunya sih kos khusus cew...,aku dulunya sih kos khusus cewe tapi pas dah ...
4,@yjnluvv baiklah :( besok ada kuliah luring ga...,@yjnluvv baiklah :( besok ada kuliah luring ga...,baiklah besok ada kuliah luring ga sih
...,...,...,...
3944,Kuliah daring kok malah lebih banyak ya tugasn...,Kuliah daring kok malah lebih banyak ya tugasn...,kuliah daring kok malah lebih banyak ya tugasn...
3945,"Kuliah media daring,\nTidak senikmat kuliah lu...","Kuliah media daring,\nTidak senikmat kuliah lu...",kuliah media daring tidak senikmat kuliah luri...
3946,"selama daring, waktunya beberes hal-hal yang d...","selama daring, waktunya beberes hal-hal yang d...",selama daring waktunya beberes hal hal yang di...
3947,"Kalo menurut sy, ketika model kuliah luring di...","Kalo menurut sy, ketika model kuliah luring di...",kalo menurut sy ketika model kuliah luring dig...


In [ ]:
# Membaca data dari file Excel
df_slang = pd.read_excel('kamusslangindonesia.xlsx')
df_slang

,slang,formal
0,woww,wow
1,aminn,amin
2,met,selamat
3,netaas,menetas
4,keberpa,keberapa
...,...,...
4416,smenjak,semenjak
4417,gmpang,gampang
4418,dia rmh,dia ramah
4419,sapa,siapa


In [ ]:
slang_to_formal = dict(zip(df_slang['slang'], df_slang['formal']))

# Fungsi untuk menerjemahkan kalimat slang ke bahasa baku
def translate_to_baku(kalimat, slang_dict):
    words = kalimat.split()
    translated_words = [slang_dict.get(word, word) for word in words]
    translated_sentence = ' '.join(translated_words)
    return translated_sentence

In [ ]:
# Terjemahkan setiap kalimat slang
df['kalimat_baku'] = df['cleanData'].apply(lambda x: translate_to_baku(x, slang_to_formal))

# Tampilkan hasil dalam DataFrame
df_hasil = df[['cleanData', 'kalimat_baku']]
df_hasil = df_hasil.rename(columns={'cleanData': 'slang_sentence', 'kalimat_baku': 'standard_sentence'})
# Print hasil
df_hasil

,slang_sentence,standard_sentence
0,apa saja keunikan dari unpad dan jatinangor h...,apa saja keunikan dari unpad dan jatinangor ha...
1,nek ono kuliah luring ngene tapi aku ngantuk b...,nek sono kuliah luring ngene tapi aku ngantuk ...
2,banyak kuliah luring ngeluh banyak kuliah dari...,banyak kuliah luring ngeluh banyak kuliah dari...
3,aku dulunya sih kos khusus cewe tapi pas dah ...,aku dulunya sih kos khusus cewek tapi pas deh ...
4,baiklah besok ada kuliah luring ga sih,baiklah besok ada kuliah luring tidak sih
...,...,...
3944,kuliah daring kok malah lebih banyak ya tugasn...,kuliah daring kok malah lebih banyak ya tugasn...
3945,kuliah media daring tidak senikmat kuliah luri...,kuliah media daring tidak senikmat kuliah luri...
3946,selama daring waktunya beberes hal hal yang di...,selama daring waktunya beberes hal hal yang di...
3947,kalo menurut sy ketika model kuliah luring dig...,kalau menurut saya ketika model kuliah luring ...


In [ ]:
# Function to clean the standard_sentence using regex
def clean_standard_sentence(sentence):
    cleaned_sentence = re.compile(r'(.)\1{1,}', re.IGNORECASE).sub(r'\1', sentence)
    return cleaned_sentence

# Apply the cleaning function to the standard_sentence column
df_hasil['standard_sentence'] = df_hasil['standard_sentence'].apply(clean_standard_sentence)

# Print the cleaned DataFrame
print(df_hasil)

csv_file_path = 'file.csv'

# Export the DataFrame to a CSV file
df_hasil.to_csv(csv_file_path, index=False)


                                         slang_sentence  \
0      apa saja keunikan dari unpad dan jatinangor h...   
1     nek ono kuliah luring ngene tapi aku ngantuk b...   
2     banyak kuliah luring ngeluh banyak kuliah dari...   
3      aku dulunya sih kos khusus cewe tapi pas dah ...   
4               baiklah besok ada kuliah luring ga sih    
...                                                 ...   
3944  kuliah daring kok malah lebih banyak ya tugasn...   
3945  kuliah media daring tidak senikmat kuliah luri...   
3946  selama daring waktunya beberes hal hal yang di...   
3947  kalo menurut sy ketika model kuliah luring dig...   
3948   percayalah kuliah daring lebih menyulitkan dr...   

                                      standard_sentence  
0     apa saja keunikan dari unpad dan jatinangor ha...  
1     nek sono kuliah luring ngene tapi aku ngantuk ...  
2     banyak kuliah luring ngeluh banyak kuliah dari...  
3     aku dulunya sih kos khusus cewek tapi pas deh ...  
4

In [ ]:
import pandas as pd

df.to_csv('datasetubah.xlsx', sep=',', index=False)

In [ ]:
import torch
from sklearn.model_selection import train_test_split
from transformers import MBart50Tokenizer, MBartForConditionalGeneration, Trainer, TrainingArguments
import pandas as pd

#Load data split it into training and validation sets
data = pd.read_csv('file.csv')
slang_sentences = data['slang_sentence'].tolist()
standard_sentences = data['standard_sentence'].tolist()

# Split data
slang_train, slang_val, standard_train, standard_val = train_test_split(
    slang_sentences, standard_sentences, test_size=0.1, random_state=42)

# Initialize the tokenizer
tokenizer = MBart50Tokenizer.from_pretrained("facebook/mbart-large-50")

# Tokenize the data
train_encodings = tokenizer(slang_train, truncation=True, padding=True, return_tensors="pt")
train_labels = tokenizer(standard_train, truncation=True, padding=True, return_tensors="pt").input_ids
val_encodings = tokenizer(slang_val, truncation=True, padding=True, return_tensors="pt")
val_labels = tokenizer(standard_val, truncation=True, padding=True, return_tensors="pt").input_ids

# Custom dataset
class TranslationDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item["labels"] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.encodings.input_ids)

train_dataset = TranslationDataset(train_encodings, train_labels)
val_dataset = TranslationDataset(val_encodings, val_labels)

# Initialize the model
model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50")

# Training arguments
training_args = TrainingArguments(
    output_dir="./translation_model",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
    save_steps=100,
    save_total_limit=2,
    remove_unused_columns=False,
    logging_steps=10,
    logging_dir="./logs",
)

# Custom Trainer
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs):
        labels = inputs["labels"]
        outputs = model(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"], labels=labels)
        return outputs.loss

# membuat custom trainer
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

# Train the model
trainer.train()

tokenizer_config.json:   0%|          | 0.00/531 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/261 [00:00<?, ?B/s]

Step,Training Loss
10,9.697200
20,7.243500
30,5.306900
40,3.588900
50,1.699500
60,0.989200
70,0.545800
80,0.422200
90,0.294200
100,0.435200


Step,Training Loss
10,9.697200
20,7.243500
30,5.306900
40,3.588900
50,1.699500
60,0.989200
70,0.545800
80,0.422200
90,0.294200
100,0.435200


TrainOutput(global_step=890, training_loss=1.0554770136147402, metrics={'train_runtime': 12947.0723, 'train_samples_per_second': 0.549, 'train_steps_per_second': 0.069, 'total_flos': 1233520105488384.0, 'train_loss': 1.0554770136147402, 'epoch': 2.0})

In [ ]:
# Save the fine-tuned model
model.save_pretrained("model_datasetLengkap")
tokenizer.save_pretrained("model_datasetLengkap")

('model_datasetLengkap/tokenizer_config.json',
 'model_datasetLengkap/special_tokens_map.json',
 'model_datasetLengkap/sentencepiece.bpe.model',
 'model_datasetLengkap/added_tokens.json')

In [ ]:
train_encodings
train_labels

tensor([[250004,    452,  50232,  ...,      1,      1,      1],
        [250004,   5117,   1733,  ...,      1,      1,      1],
        [250004,   4591,   3972,  ...,      1,      1,      1],
        ...,
        [250004,  27547,  49985,  ...,      1,      1,      1],
        [250004,    151, 154968,  ...,      1,      1,      1],
        [250004,  19485,     34,  ...,      1,      1,      1]])

tensor([[250004,    452,  50232,  ...,      1,      1,      1],
        [250004,   5117,   1733,  ...,      1,      1,      1],
        [250004,   4591,   3972,  ...,      1,      1,      1],
        ...,
        [250004,  27547,  49985,  ...,      1,      1,      1],
        [250004,    151, 154968,  ...,      1,      1,      1],
        [250004,  19485,     34,  ...,      1,      1,      1]])

In [ ]:
def translate_sentence(sentence):
    inputs = tokenizer([sentence], return_tensors="pt")
    outputs = model.generate(**inputs)
    translated_sentence = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return translated_sentence


slang_sentence = "bisa ga sih kuliah luring tpi uts uas ny daring trs"
translated_sentence = translate_sentence(slang_sentence)

print("Slang sentence:      ", slang_sentence)
print("Translated sentence: ", translated_sentence)

Slang sentence:       bisa ga sih kuliah luring tpi uts uas ny daring trs
Translated sentence:  bisa tidak sih kuliah luring tapi uts uas nya daring terus


In [ ]:
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction


# Reference sentence (corresponding standard sentence)
reference_sentence = "bisa tidak sih kuliah luring tapi uts uas nya daring terus"
print("Slang sentence:      ", slang_sentence)
print("reference_sentence:  ", reference_sentence )
print("Translated sentence: ", translated_sentence)
# Function to compute BLEU score with smoothing
def compute_bleu_score(reference, translation):
    smoothie = SmoothingFunction().method4
    return sentence_bleu([reference], translation, smoothing_function=smoothie)

# Compute BLEU score

bleu_score = compute_bleu_score(reference_sentence, translated_sentence)
print("BLEU Score:", bleu_score)

Slang sentence:       bisa ga sih kuliah luring tpi uts uas ny daring trs
reference_sentence:   bisa tidak sih kuliah luring tapi uts uas nya daring terus
Translated sentence:  bisa tidak sih kuliah luring tapi uts uas nya daring terus
BLEU Score: 1.0
